In [1]:
import pandas as pd
import numpy as np
import json 
import networkx as nx 
import linkpred

In [2]:
with open('../data_collection/data/tuscany_init.json') as jf:
    data = json.load(jf)
    
len(data)

15863

In [3]:
es = {'a':1,'b':2,'c':3}

In [7]:
for i in range(9): 
    if i not in es: es[i] = 784387

In [ ]:
keys  = list(data.keys())

tuscany = dict()

In [13]:
tuscany = dict()

for restaurant, reviews in data.items():
    if len(reviews)>0 : tuscany[restaurant] = dict()
    else: continue
    for review in reviews : 
        username = review['username']
        date = review['date']
        
        if username not in tuscany[restaurant] : 
            tuscany[restaurant][username] = [date,date]
        else:
            dates = tuscany[restaurant][username]
            if date > dates[1]: tuscany[restaurant][username][1] = date 
            if date < dates[0]: tuscany[restaurant][username][0] = date 

In [11]:
tup = (1,2,3)

In [12]:
tup[2] = 'cacc'

TypeError: 'tuple' object does not support item assignment

In [16]:
with open('../data_collection/data/tuscany_firstlast.json','w+') as jf:
    json.dump(tuscany,jf)

In [17]:
tu

15863

In [28]:
tuscanybig = tuscany

In [42]:
tuscany2014 = dict()
for restaurant, reviews in tuscany.items():
    if len(reviews)>0 :
        tuscany2014[restaurant] = dict()
    else: continue
    for reviewer,dates in reviews.items() : 
        if int(dates[0]) <20140000:
            tuscany2014[restaurant][reviewer] = dates

In [43]:
len(tuscany2014)

15863

In [44]:
tuscany = tuscany2014

In [51]:
G = nx.Graph()

In [52]:
for i,reviews_i in tuscany.items():
    if int(i) % 1000 == 0: print(i, end= ' ')
    if len(tuscany[str(i)])==0: continue
    reviewers_i = set(reviews_i.keys())

    for j, reviews_j in list(tuscany.items())[int(i)+1:]:
        if len(tuscany[str(j)])==0: continue

        reviewers_j = set(reviews_j.keys())
        common_reviewers = dict()
        intersection = list(reviewers_i.intersection(reviewers_j))
        if len(intersection) > 0:
            for reviewer in intersection:
                dates_i = reviews_i[reviewer]
                dates_j = reviews_j[reviewer]
                common_reviewers[reviewer] = [max(dates_i[0],dates_j[0]),min(dates_i[1],dates_j[1])] 

            first = min([review[0] for review in common_reviewers.values()])
            last = max([review[1] for review in common_reviewers.values()])
            G.add_edge(i,j, first=first,last=last)


0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 

In [53]:
nx.write_graphml(G,'pre2014.graphml')

In [54]:
len(G)

5796

In [63]:
len(G.edges)*0.7

256891.59999999998

In [68]:
training = [e for e in list(G.edges) if int(G.edges[e]['first']) <20130815]

In [69]:
len(training)

254531

In [70]:
g = G.edge_subgraph(training)

In [73]:
jc = linkpred.predictors.Jaccard(g, excluded=g.edges())
jc_results = jc.predict()

top = jc_results.top(5)
for edge, score in top.items():
    print(edge, score)

KeyboardInterrupt: 

In [ ]:
test = G.edge_subgraph([e for e in G.edges() if e not in training])

In [ ]:
import random
import itertools
from linkpred.evaluation import Pair

universe = set([Pair(i) for i in itertools.product(nodes, nodes) if i[0]!=i[1]])

In [ ]:
jc_evaluation = linkpred.evaluation.EvaluationSheet(jc_results, test, universe)